In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

In [ ]:
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)
model.to("cuda:0")

In [ ]:
# url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
image = Image.open("./assets/palm_tree_og.png")
display(image)

In [ ]:
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Describe the content, composition and style of this image with one short sentence for each. Use about 40 words."},
        ],
    }
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=False) + " The image is"
inputs = processor(image, prompt, return_tensors="pt").to("cuda:0")
print(prompt)

In [ ]:
output = model.generate(**inputs, max_new_tokens=300)
decoded = processor.decode(output[0], skip_special_tokens=True)
decoded = decoded.split("The image is ")[1]
wordcount = len(decoded.split())
print(f"Wordcount: {wordcount}")
print(decoded)

In [ ]:
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Please identify objects with significant shadow in the image. Format your response as a JSON list of nouns."},
        ],
    }
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=False) + " ["
inputs = processor(image, prompt, return_tensors="pt").to("cuda:0")
print(prompt)

In [ ]:
output = model.generate(**inputs, max_new_tokens=300)
decoded = processor.decode(output[0], skip_special_tokens=True)
decoded = decoded.split("[/INST] ")[1]
wordcount = len(decoded.split())
print(f"Wordcount: {wordcount}")
print(decoded)

In [ ]:
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Describe the background of the image. Use about 40 words."},
        ],
    }
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=False) + " The background "
inputs = processor(image, prompt, return_tensors="pt").to("cuda:0")
output = model.generate(**inputs, max_new_tokens=300)
decoded = processor.decode(output[0], skip_special_tokens=True)
decoded = decoded.split("[/INST] ")[1]
wordcount = len(decoded.split())
print(f"Wordcount: {wordcount}")
print(decoded)